<a href="https://colab.research.google.com/github/dp2292/CP01-Data-Alchemists/blob/main/Neural_Network_CP01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data

In [2]:
!gdown --id 1RfhNYBd7g70LdgJROvx8iPGygPhiGQLZ -O data/

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RfhNYBd7g70LdgJROvx8iPGygPhiGQLZ
To: /content/data/CP01.zip
100% 630M/630M [00:13<00:00, 45.1MB/s]


In [3]:
!unzip /content/data/CP01.zip -d /content/data

Archive:  /content/data/CP01.zip
  inflating: /content/data/For_modeling.csv  
  inflating: /content/data/cleaned_seoul_bike_data.csv  


In [4]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('data/For_modeling.csv',index_col=0)
data = data.reset_index().drop(columns=['index'])
data.head(5)

,Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,Phour,...,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust
0,3,50,37.544666,126.888359,37.544666,126.888359,0.000000,1,1,0,...,4,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
1,24,7670,37.506199,127.003944,37.551250,127.035103,5.713529,1,1,0,...,25,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
2,8,1390,37.544590,127.057083,37.537014,127.061096,0.913702,1,1,0,...,9,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
3,8,1820,37.571102,127.023560,37.561447,127.034920,1.468027,1,1,0,...,10,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
4,4,850,37.573242,127.015907,37.565849,127.016403,0.823227,1,1,0,...,6,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0


In [5]:
filtered_data = data[data['Haversine'] != 0]

In [6]:
#Remove all duplicates
filtered_data.drop_duplicates(inplace=True)
print("Dataset size after removing duplicates:", filtered_data.shape)

Dataset size after removing duplicates: (8636811, 25)


<ipython-input-6-ca0abfc58edb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.drop_duplicates(inplace=True)


In [8]:
columns_to_check = ['Distance', 'Haversine', 'Dhour', 'Phour', 'Temp']

# Calculate the quartiles and IQR for the specified columns
Q1 = filtered_data[columns_to_check].quantile(0.25)
Q3 = filtered_data[columns_to_check].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outlier detection
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Create a boolean mask for outliers in each column
outliers_mask = ((filtered_data[columns_to_check] < lower_bound) | (filtered_data[columns_to_check] > upper_bound)).any(axis=1)

# Apply the mask to remove rows with outliers
filtered_data = filtered_data[~outliers_mask]

In [9]:
print(filtered_data.shape)

(7652385, 25)


In [7]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import numpy as np

In [ ]:
columns = ['Distance','Duration','Haversine','Phour','GroundTemp']
df = filtered_data[columns]
df.describe()
df = df.iloc[0:300000]
df.shape

(300000, 5)

In [ ]:
X_train,X_test, Y_train, Y_test = train_test_split(df.drop(columns=['Duration']), df['Duration'], test_size=0.2, random_state=43)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a neural network model
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=4))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='linear'))  # Use linear activation for regression

# Compile your model with the Mean Squared Error loss
model.compile(optimizer='adam', loss=MeanSquaredError())

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, Y_train, batch_size=10, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, Y_test)

# Calculate and print custom accuracy metric
y_pred = model.predict(X_test)

# Calculate and print training loss
train_loss = history.history['loss'][-1]

print(f"Training Loss (MSE): {train_loss:.4f}")
print(f"Test Loss (MSE): {test_loss:.4f}")

Epoch 1/100
19200/19200 [==============================] - 73s 3ms/step - loss: 128.5794 - val_loss: 124.0492
Epoch 2/100
19200/19200 [==============================] - 67s 3ms/step - loss: 123.3661 - val_loss: 123.7404
Epoch 3/100
19200/19200 [==============================] - 67s 3ms/step - loss: 122.9846 - val_loss: 122.5465
Epoch 4/100
19200/19200 [==============================] - 70s 4ms/step - loss: 122.5905 - val_loss: 122.6681
Epoch 5/100
19200/19200 [==============================] - 70s 4ms/step - loss: 122.3910 - val_loss: 122.0466
Epoch 6/100
19200/19200 [==============================] - 67s 3ms/step - loss: 122.3680 - val_loss: 122.2364
Epoch 7/100
19200/19200 [==============================] - 70s 4ms/step - loss: 122.2884 - val_loss: 122.5774
Epoch 8/100
19200/19200 [==============================] - 67s 3ms/step - loss: 122.0717 - val_loss: 122.2210
Epoch 9/100
19200/19200 [==============================] - 67s 3ms/step - loss: 122.0751 - val_loss: 125.4857
Epoch 10/1